In [2]:
using CUDA
using DelimitedFiles
include("../../src/forces_func.jl")
include("../../src/struct_data.jl")

SumAgg (generic function with 1 method)

In [3]:
@make_struct_func Cubic

force_func (generic function with 1 method)

## Add Data

In [4]:
parameters = Model(
    Cubic(
        μ₁      = 0.1,
        rₘᵢₙ    = 2,
        rₘₐₓ    = 3
    ),
    Contractile(
        fₚ      = 0.0000001
    ),
    Time(
        t_f     = 100000, 
        dt      = 0.1
    ),
    Neighbor(
        n_knn   = 50,
        nn      = 50
    ),
    Geometry(
        R_agg   = 9,
        position = [
            1 0 0;
            0 0 0
        ]
    ),
    Simulation(
        n_text  = 200,
        path    = "BBB.xyz"
    )
)

Model(Cubic(0.1, 2.0, 3.0), Contractile(1.0e-7), Time(100000.0, 0.1), Neighbor(50, 12), Geometry(9.0, [1 0 0; 0 0 0]), Simulation(200, "BBB.xyz"))

In [5]:
X = Float32.(readdlm("../../data/init/Sphere/$(Int(parameters.Geometry.R_agg)).xyz")[3:end,2:end]) |> cu
Agg = Aggregate(Neighbor(50, 12), PositionCell(X));

In [21]:
# Variables for iterator
it_force = Cubic(
    μ₁      = 0.1,
    rₘᵢₙ    = 2,
    rₘₐₓ    = 3
)
it_contractile = Contractile(
    fₚ      = 0.0000001
)

## Var before Iterator
force(r) =  force_func(it_force::Cubic, r)

force (generic function with 1 method)

## neighbor.jl -> cu_knn()

In [10]:
function cu_knn(Agg::Aggregate)
    # Definig Variables for calculing knn
    global Agg
    
    # Defining Coordinates of each cell on the aggregates
    Agg.Neighbor.i_Cell = reshape(
                repeat(
                    Agg.Position.X, 
                    size(Agg.Position.X ,1)
                ), 
                size(Agg.Position.X ,1), 
                size(Agg.Position.X ,1), 
                3
            ) - 
            reshape(
                repeat(
                    Agg.Position.X, 
                    inner=(size(Agg.Position.X ,1),1)
                ), 
                size(Agg.Position.X ,1), 
                size(Agg.Position.X ,1), 
                3
            )

    # Calculating Norm on every cell on the aggregate
    Agg.Neighbor.Dist = sqrt.(
                Agg.Neighbor.i_Cell[:,:,1] .^ 2 + 
                Agg.Neighbor.i_Cell[:,:,2] .^ 2 + 
                Agg.Neighbor.i_Cell[:,:,3] .^ 2
                )
    # # i_Cell = nothing; GC.gc(true)

    # Calculating index of knof each cell in the aggregate
    for i = 1:Agg.ParNeighbor.nn
        Agg.Neighbor.idx[i,:] = findmin(Agg.Neighbor.Dist; dims=1)[2]
        Agg.Neighbor.Dist[Agg.Neighbor.idx[i,:]] .= Inf
    end
    synchronize()
end

cu_knn (generic function with 1 method)

In [13]:
CUDA.@time cu_knn(Agg)
Agg.Neighbor.idx

  0.011886 seconds (3.38 k CPU allocations: 270.250 KiB) (60 GPU allocations: 20.244 MiB, 13.14% memmgmt time)


12×538 CuArray{CartesianIndex{2}, 2, CUDA.Mem.DeviceBuffer}:
 CartesianIndex(1, 1)   CartesianIndex(2, 2)   …  CartesianIndex(538, 538)
 CartesianIndex(26, 1)  CartesianIndex(27, 2)     CartesianIndex(513, 538)
 CartesianIndex(3, 1)   CartesianIndex(4, 2)      CartesianIndex(534, 538)
 CartesianIndex(4, 1)   CartesianIndex(5, 2)      CartesianIndex(535, 538)
 CartesianIndex(30, 1)  CartesianIndex(31, 2)     CartesianIndex(507, 538)
 CartesianIndex(31, 1)  CartesianIndex(32, 2)  …  CartesianIndex(508, 538)
 CartesianIndex(2, 1)   CartesianIndex(1, 2)      CartesianIndex(537, 538)
 CartesianIndex(25, 1)  CartesianIndex(26, 2)     CartesianIndex(512, 538)
 CartesianIndex(27, 1)  CartesianIndex(28, 2)     CartesianIndex(514, 538)
 CartesianIndex(37, 1)  CartesianIndex(38, 2)     CartesianIndex(501, 538)
 CartesianIndex(77, 1)  CartesianIndex(78, 2)  …  CartesianIndex(461, 538)
 CartesianIndex(5, 1)   CartesianIndex(3, 2)      CartesianIndex(533, 538)

## run_event.jl -> simulate() -> rand_idx

In [15]:
Agg.Neighbor.rand_idx = getindex.(
                            Agg.Neighbor.idx, 1
                            )[getindex.(rand(
                                    2:Agg.ParNeighbor.nn,
                                    2*size(Agg.Position.X,1)
                                ),1),:]

1076×538 CuArray{Int64, 2, CUDA.Mem.DeviceBuffer}:
 26  27  30  31  32  33  43  44  45  46  …  496  506  507  508  509  512  513
 26  27  30  31  32  33  43  44  45  46     496  506  507  508  509  512  513
  4   5  36   2  38  40  12  13  14  15     527  499  538  501  503  534  535
 30  31  37  37  39   5  13  14  15  16     531  500  500  502  535  506  507
  4   5  36   2  38  40  12  13  14  15     527  499  538  501  503  534  535
 30  31  37  37  39   5  13  14  15  16  …  531  500  500  502  535  506  507
  5   3  84  32  46  28  91  36  37  38     488  453  508  495  514  535  533
  5   3  84  32  46  28  91  36  37  38     488  453  508  495  514  535  533
 30  31  37  37  39   5  13  14  15  16     531  500  500  502  535  506  507
 25  26   8   5   9  32   3  37  38  39     489  529  535  530  508  511  512
 26  27  30  31  32  33  43  44  45  46  …  496  506  507  508  509  512  513
 31  32   4  38   4  10   8   7   8   9     503  534  501  534  531  507  508
 25  26   8  

## forces.jl -> cu_forces

In [22]:
function cu_force(t::Time, f::Contractile, Agg::Aggregate)
    # Definig Variables for calculing dX
    global Agg

    # Calculating distance for random forces (contractile)
    Agg.Force.r_p = Agg.Position.X .- 
                        Agg.Position.X[
                            Agg.Neighbor.rand_idx[
                                Int.(mod(
                                    Agg.t, size(Agg.Position.X, 1)
                                ) .+ 1),
                            :],
                        :]
    
    # Finding Distances/Norm for random forces
    Agg.Force.dist_p = sum(Agg.Force.r_p .^ 2, dims=2).^ 0.5

    # Finding distances
    Agg.Force.r = reshape(
            repeat(Agg.Position.X, inner=(Agg.ParNeighbor.nn,1)), 
            Agg.ParNeighbor.nn, size(Agg.Position.X)[1], 3
        ) .- 
        Agg.Position.X[getindex.(Agg.Neighbor.idx,1),:]

    # Finding Distances(Norm)
    Agg.Force.dist = ((sum(Agg.Force.r .^ 2, dims=3)) .^ 0.5)[:,:,1]

    # # Finding forces for each cell
    Agg.Force.F = force(Agg.Force.dist) .* Agg.Force.r ./ Agg.Force.dist

    # # Calculating de dX   -> dX[i,:] +=  r/dist * F
    Agg.Position.dX = sum(Agg.Force.F[2:end,:,:]; dims=1)[1,:,:] -                                       
                        f.fₚ .* (Agg.Force.r_p ./ Agg.Force.dist_p)
    synchronize()
end

cu_force (generic function with 1 method)

In [23]:
cu_force(
    Time(100000, 0.1),
    it_contractile,
    Agg
)
Agg.Position.dX

538×3 CuArray{Float64, 2, CUDA.Mem.DeviceBuffer}:
  0.0          -0.000149285   0.00364663
  0.0          -0.000149285   0.00364663
 -8.91946e-5    0.00149167    0.00364662
  5.42101e-20  -0.000372395   0.0022224
  8.91946e-5    0.00149167    0.00364662
  0.0           0.00133737    0.00364662
  0.000338466  -0.000998528   0.00364838
 -0.00171622   -0.000592169   0.0022497
 -0.00171622   -0.000592169   0.0022497
 -0.00171622   -0.000592169   0.0022497
 -0.000249384  -0.000844242   0.00364838
 -0.000178415  -0.000271155   0.00352155
 -0.00171622    0.000724223   0.00212288
  ⋮                          
 -5.01141e-8    0.000271155  -0.00352155
  0.000338466   0.000998528  -0.00364838
 -0.00171622    0.000592169  -0.0022497
 -0.00171622    0.000592169  -0.0022497
 -0.00171622    0.000592169  -0.0022497
 -0.000249384   0.000844242  -0.00364838
 -8.91946e-5   -0.00149167   -0.00364662
  5.42101e-20   0.000372395  -0.0022224
  8.91946e-5   -0.00149167   -0.00364662
  0.0          -0.00133737